##  Oil Blending

Sunco Oil manufactures three types of gasoline (gas-1, gas-2, gas-3). Each type of gasoline is produced by blending three types of crude oil (crude-1, crude-2, crude-3). Details are as follows:

- Purchase prices for each type of crude oil:

|Type | Price/Barrel |
|:-----------:|:---------------------------:|
| crude-1 | \$45 |
| crude-2 | \$35 |
| crude-3 | \$25 |

- Sale prices of each type of gasoline:

|Type | Price/Barrel | 
|:---------:|:------------------------:|
|gas-1 | \$70 |
|gas-2 | \$60 |
|gas-3 | \$50 | 

- Sunco can purchase up to 5000 barrels of each type of crude oil daily, and their refinery can produce up to 14000 barrels of gasoline daily.
- It costs $4 to transform one barrel of oil into one barrel of gasoline.
  
- The 3 types of crude oil have different octane levels and sulfur contents:

|Type | Octane | Sulfur (%) |
|:----:|:-----:|:---:|
| crude-1 | 12 | 0.5 |
| crude-2 | 6 | 2.0 |
| crude-3 | 8 | 3.0 |

- When different types of crude oil are mixed, octane levels and sulfur contents blend linearly.

- Each type of gasoline has different requirements for **average** octane level and sulfur content:
    - Average octane levels of **at least** 10 for gas-1, at least 8 for gas-2, and at least 6 for gas-3.
    - Average sulfur content of **at most** 1\% for gas-1, at most 2\% for gas-2, and at most 1\% for gas-3.
<br/><br/>

- Sunco is contractually obligated to supply (at a minimum) the following amounts of each type of gasoline:

|Type | Barrels/day|
|:---------:|:---------:|
| gas-1 | 3000 |
| gas-2 | 2000 |
| gas-3 | 1000 |

**Goal:** Devise a production to maximize daily profit (revenue $-$ cost) while meeting supply, production, and demand requirements.

To solve this problem, we use decision variables $x_{ij}$ which represent the number of barrels of crude-$i$ used to produce gas-$j$.

We organize the data in terms of the type of crude oil and the type of gasoline as follows.

In [ ]:
#Types of crude oil and gas stored as vectors of symbols
crudetypes = [:crude_1, :crude_2, :crude_3]
gastypes = [:gas_1, :gas_2, :gas_3]

#Dictionaries for purchase prices of each crude type, sale prices of each gas type
purchaseprices = Dict(zip(crudetypes, [45, 35, 25]))
saleprices = Dict(zip(gastypes, [70, 60, 50]))

#Dictionaries for octane levels and sulfur contents of each crude oil type
octanecrude = Dict(zip(crudetypes, [12,6,8]))
sulfurcrude = Dict(zip(crudetypes, [0.005,0.02,0.03]))

#Dictionaries for minimum average octane levels, maximum average sulfur levels in each type of gasoline
octanegas = Dict(zip(gastypes, [10, 8, 6]))
sulfurgas = Dict(zip(gastypes, [0.01, 0.02, 0.01]))

#Dictionary for demand of each type of gasoline
demands = Dict(zip(gastypes, [3000, 2000, 1000]))

#Daily purchase limits (in barrels) on each type of crude oil
supplies = Dict(zip(crudetypes, [5000, 5000, 5000]))

#Cost to transform one barrel of crude into one barrel of gasoline
transformcost = 4
#Maximum gasoline production per day (in barrels)
maxproduction = 14000;

Now, we can create our model.

In [ ]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

@variable(m, x[crudetypes, gastypes] >= 0)

#Maximum amount produced per day
@constraint(m, mprodconstraint, sum(sum(x[i,j] for j in gastypes) for i in crudetypes) <= maxproduction)

#Maximum amounts of each type of crude that can be purchased
@constraint(m, mpurconstraint[i in crudetypes], sum(x[i,j] for j in gastypes) <= supplies[i])

#Meet demand obligations
@constraint(m, meetdemand[j in gastypes], sum(x[i,j] for i in crudetypes) >= demands[j])

#Minimum average octane constraint
@constraint(m, minoctane[j in gastypes], sum(octanecrude[i]*x[i,j] for i in crudetypes)
                                        >= sum(octanegas[j]*x[i,j] for i in crudetypes))
#Maximum average sulfur constraint
@constraint(m, maxsulfur[j in gastypes], sum(sulfurcrude[i]*x[i,j] for i in crudetypes)
                                        <= sum(sulfurgas[j]*x[i,j] for i in crudetypes))

#Objective is revenue (sales) minus cost (purchase and conversion)
@objective(m, Max, sum(saleprices[j]*sum(x[i,j] for i in crudetypes) for j in gastypes)
    - sum((purchaseprices[i]+transformcost)*sum(x[i,j] for j in gastypes) for i in crudetypes))

print(m)

In [ ]:
#Solve model and store optimal objective and variable values
optimize!(m)
blend = value.(x)
objval = objective_value(m)

#Print status of model
println("\nSolver terminated with status ",termination_status(m))

#Only print out optimal solution if there is one
if is_solved_and_feasible(m)
    for j in gastypes
        println("\nBlend")
        for i in crudetypes
            println("\r",round(blend[i,j])," barrels of ",i,", ")
        end
        println("\ninto ",sum(blend[i,j] for i in crudetypes)," barrels of ",j,".")
    end
end

println("\nMaximum daily profit: \$",objval)

In [ ]:
#Generate sensitivity report
report = lp_sensitivity_report(m)

In [ ]:
x33_range = report[x[:crude_3,:gas_3]]
println("c[:crude_3,:gas_3] can stay between ", coefficient(objective_function(m), x[:crude_3,:gas_3])+x33_range[1], 
    " and ", coefficient(objective_function(m), x[:crude_3,:gas_3])+x33_range[2])

gas1_range = report[meetdemand[:gas_1]]
println("Demand for gas-1 can stay between ", demands[:gas_1]+gas1_range[1], " and ", demands[:gas_1]+gas1_range[2])